In [ ]:
def main():
    """Main function to demonstrate the workflow with Gemini integration"""
    
    # Initialize Gemini (you need to set your API key)
    try:
        print("🔑 Initializing Gemini AI...")
        # You can either set the GOOGLE_API_KEY environment variable
        # or pass it directly: initialize_gemini("your_api_key_here")
        initialize_gemini()
        print("✅ Gemini AI initialized successfully!")
    except Exception as e:
        print(f"❌ Failed to initialize Gemini: {e}")
        print("💡 Make sure to set GOOGLE_API_KEY environment variable or pass API key directly")
        return
    
    # Create the application
    app = create_fitness_app_graph()
    
    # Initialize state
    initial_state = FitnessAppState(
        height=None,
        weight=None,
        age=None,
        gender=None,
        activity_level=None,
        goal_type=None,
        target_weight=None,
        target_days=None,
        user_notes=None,
        nutrition_plan=None,
        nutrition_plan_approved=None,
        workout_plan=None,
        current_step="start",
        user_input_pending=False,
        edit_request=None,
        chat_messages=[],
        chat_query=None,
        chat_response=None,
        error_message=None
    )
    
    # Configuration for conversation persistence
    config = RunnableConfig(configurable={"thread_id": "user_123"})
    
    # Example: Start the workflow
    print("\n🚀 Starting Fitness AI Application with Gemini")
    print("=" * 60)
    
    # Step 1: Collect personal info
    result = app.invoke(initial_state, config)
    print(f"Current step: {result['current_step']}")
    
    # Example of simulating user input and continuing workflow
    print("\n📋 Simulating user data input...")
    updated_state = result.copy()
    updated_state.update({
        "height": 175.0,
        "weight": 80.0,
        "age": 28,
        "gender": "male",
        "activity_level": "moderately_active",
        "user_input_pending": False
    })
    
    # Continue to goals collection
    result = app.invoke(updated_state, config)
    print(f"Current step: {result['current_step']}")
    
    # Add goals
    print("\n🎯 Adding fitness goals...")
    updated_state = result.copy()
    updated_state.update({
        "goal_type": "weight_loss",
        "target_weight": 75.0,
        "target_days": 90,
        "user_input_pending": False
    })
    
    # Continue to notes collection
    result = app.invoke(updated_state, config)
    print(f"Current step: {result['current_step']}")
    
    # Add notes
    print("\n📝 Adding user notes...")
    updated_state = result.copy()
    updated_state.update({
        "user_notes": "I'm vegetarian, prefer home workouts, and have a busy schedule. I also have a mild lactose intolerance.",
        "user_input_pending": False
    })
    
    # Generate nutrition plan with Gemini
    print("\n🍎 Generating nutrition plan with Gemini AI...")
    result = app.invoke(updated_state, config)
    
    if result.get("nutrition_plan"):
        print("✅ Nutrition plan generated!")
        print("📋 Plan preview:", str(result["nutrition_plan"])[:200] + "...")
    
    # Simulate approving nutrition plan
    print("\n✅ Simulating nutrition plan approval...")
    updated_state = result.copy()
    updated_state.update({
        "nutrition_plan_approved": True,
        "user_input_pending": False
    })
    
    # Generate workout plan with Gemini
    print("\n💪 Generating workout plan with Gemini AI...")
    result = app.invoke(updated_state, config)
    
    if result.get("workout_plan"):
        print("✅ Workout plan generated!")
        print("📋 Plan preview:", str(result["workout_plan"])[:200] + "...")
    
    # Example chat interaction
    print("\n💬 Testing chat functionality...")
    chat_state = result.copy()
    chat_state.update({
        "chat_query": "Can you suggest some quick breakfast options that fit my nutrition plan and dietary restrictions?"
    })
    
    chat_result = app.invoke(chat_state, config)
    if chat_result.get("chat_response"):
        print("🤖 Chat response:", chat_result["chat_response"][:300] + "...")
    
    print("\n" + "=" * 60)
    print("🎉 Workflow demonstration completed successfully!")
    print("\nKey features demonstrated:")
    print("✅ Gemini AI integration for all plan generation")
    print("✅ Sequential data collection workflow")
    print("✅ Personalized nutrition plan generation")
    print("✅ Personalized workout plan generation")
    print("✅ Contextual chat support")
    print("✅ State persistence across interactions")
    print("✅ Error handling and fallbacks")
    
    print("\n🔧 Setup Requirements:")
    print("1. Install: pip install google-generativeai langgraph langchain-core")
    print("2. Get Google API key from: https://makersuite.google.com/app/apikey")
    print("3. Set environment variable: export GOOGLE_API_KEY='your_key_here'")
    print("4. Or pass directly: initialize_gemini('your_key_here')")

# Additional utility functions for production use
def save_user_state(user_id: str, state: FitnessAppState):
    """Save user state to persistent storage"""
    # Implementation would depend on your storage solution (Redis, DB, etc.)
    filename = f"user_state_{user_id}.json"
    with open(filename, 'w') as f:
        # Convert state to serializable format
        serializable_state = {k: v for k, v in state.items() if v is not None}
        json.dump(serializable_state, f, indent=2)
    print(f"💾 User state saved to {filename}")

def load_user_state(user_id: str) -> Optional[FitnessAppState]:
    """Load user state from persistent storage"""
    filename = f"user_state_{user_id}.json"
    try:
        with open(filename, 'r') as f:
            data = json.load(f)
            return FitnessAppState(**data)
    except FileNotFoundError:
        print(f"⚠️ No saved state found for user {user_id}")
        return None

def create_web_interface():
    """Example of how you might structure a web interface"""
    interface_example = """
    # Example Flask/FastAPI integration
    
    from flask import Flask, request, jsonify
    
    app = Flask(__name__)
    fitness_graph = create_fitness_app_graph()
    
    @app.route('/api/fitness/start', methods=['POST'])
    def start_fitness_journey():
        user_id = request.json.get('user_id')
        config = RunnableConfig(configurable={"thread_id": user_id})
        
        # Initialize or load user state
        state = load_user_state(user_id) or FitnessAppState(...)
        
        result = fitness_graph.invoke(state, config)
        save_user_state(user_id, result)
        
        return jsonify(result)
    
    @app.route('/api/fitness/chat', methods=['POST'])
    def chat_with_ai():
        user_id = request.json.get('user_id')
        query = request.json.get('query')
        
        state = load_user_state(user_id)
        state['chat_query'] = query
        
        config = RunnableConfig(configurable={"thread_id": user_id})
        result = fitness_graph.invoke(state, config)
        save_user_state(user_id, result)
        
        return jsonify({'response': result['chat_response']})
    """
    print("🌐 Web interface example structure created")
    print("💡 Check the interface_example variable for Flask/FastAPI integration code")

if __name__ == "__main__":
    main()from typing import TypedDict, Optional, Dict, Any, List
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
import json

# Define the Application State
class FitnessAppState(TypedDict):
    # User Profile Information
    height: Optional[float]
    weight: Optional[float]
    age: Optional[int]
    gender: Optional[str]
    activity_level: Optional[str]
    
    # User Goals
    goal_type: Optional[str]  # e.g., "weight_loss", "muscle_gain", "maintenance"
    target_weight: Optional[float]
    target_days: Optional[int]
    
    # User Notes
    user_notes: Optional[str]
    
    # Generated Plans
    nutrition_plan: Optional[Dict[str, Any]]
    nutrition_plan_approved: Optional[bool]
    workout_plan: Optional[Dict[str, Any]]
    
    # Workflow Control
    current_step: str
    user_input_pending: bool
    edit_request: Optional[str]
    
    # Chat Context
    chat_messages: List[Dict[str, str]]
    chat_query: Optional[str]
    chat_response: Optional[str]
    
    # Error Handling
    error_message: Optional[str]

# Node Functions
def collect_personal_info(state: FitnessAppState) -> FitnessAppState:
    """Node to collect user personal information"""
    print("📋 Please provide your personal information:")
    print("- Height (in cm)")
    print("- Weight (in kg)")
    print("- Age")
    print("- Gender (male/female/other)")
    print("- Activity Level (sedentary/lightly_active/moderately_active/very_active/extremely_active)")
    
    state["current_step"] = "personal_info"
    state["user_input_pending"] = True
    return state

def collect_goals(state: FitnessAppState) -> FitnessAppState:
    """Node to collect user goals"""
    print("🎯 Please provide your fitness goals:")
    print("- Goal Type (weight_loss/muscle_gain/maintenance/strength/endurance)")
    print("- Target Weight (in kg)")
    print("- Target Timeline (in days)")
    
    state["current_step"] = "goals"
    state["user_input_pending"] = True
    return state

def collect_notes(state: FitnessAppState) -> FitnessAppState:
    """Node to collect additional user notes"""
    print("📝 Please provide any additional notes (dietary restrictions, preferences, medical conditions, etc.):")
    print("This is optional but helps create a more personalized plan.")
    
    state["current_step"] = "notes"
    state["user_input_pending"] = True
    return state

def generate_nutrition_plan(state: FitnessAppState) -> FitnessAppState:
    """Node to generate personalized nutrition plan using LLM"""
    print("🍎 Generating your personalized nutrition plan...")
    
    # Calculate BMR and daily calorie needs
    bmr = calculate_bmr(state["height"], state["weight"], state["age"], state["gender"])
    daily_calories = calculate_daily_calories(bmr, state["activity_level"])
    
    # Adjust calories based on goal
    target_calories = adjust_calories_for_goal(daily_calories, state["goal_type"], state["target_weight"], state["weight"], state["target_days"])
    
    # Create nutrition plan prompt
    nutrition_prompt = f"""
    Create a detailed nutrition plan for a user with the following profile:
    - Age: {state['age']}, Gender: {state['gender']}
    - Height: {state['height']}cm, Weight: {state['weight']}kg
    - Activity Level: {state['activity_level']}
    - Goal: {state['goal_type']}, Target Weight: {state['target_weight']}kg in {state['target_days']} days
    - Target Daily Calories: {target_calories}
    - Additional Notes: {state['user_notes']}
    
    Provide:
    1. Daily calorie breakdown (protein, carbs, fats)
    2. Meal timing suggestions
    3. Sample meal ideas for each meal
    4. Hydration recommendations
    5. Supplement suggestions (if applicable)
    """
    
    # Here you would call your LLM (OpenAI, Anthropic, etc.)
    nutrition_plan = {
        "daily_calories": target_calories,
        "macros": {
            "protein": f"{target_calories * 0.3 / 4:.0f}g",
            "carbs": f"{target_calories * 0.4 / 4:.0f}g",
            "fats": f"{target_calories * 0.3 / 9:.0f}g"
        },
        "meal_plan": "Generated meal plan based on user profile...",
        "hydration": "8-10 glasses of water daily",
        "supplements": "Based on goals and deficiencies..."
    }
    
    state["nutrition_plan"] = nutrition_plan
    state["current_step"] = "nutrition_review"
    state["user_input_pending"] = True
    return state

def handle_nutrition_edit(state: FitnessAppState) -> FitnessAppState:
    """Node to handle nutrition plan edits"""
    print("✏️ Processing your nutrition plan edits...")
    
    edit_prompt = f"""
    The user wants to modify their nutrition plan. 
    Original plan: {state['nutrition_plan']}
    Edit request: {state['edit_request']}
    User notes: {state['user_notes']}
    
    Please modify the nutrition plan according to the user's request.
    """
    
    # Here you would call your LLM to modify the plan
    # For now, we'll simulate an updated plan
    updated_plan = state["nutrition_plan"].copy()
    updated_plan["last_modified"] = "Updated based on user feedback"
    
    state["nutrition_plan"] = updated_plan
    state["current_step"] = "nutrition_review"
    state["edit_request"] = None
    return state

def generate_workout_plan(state: FitnessAppState) -> FitnessAppState:
    """Node to generate personalized workout plan"""
    print("💪 Generating your personalized workout plan...")
    
    workout_prompt = f"""
    Create a detailed workout plan for a user with the following profile:
    - Age: {state['age']}, Gender: {state['gender']}
    - Height: {state['height']}cm, Weight: {state['weight']}kg
    - Activity Level: {state['activity_level']}
    - Goal: {state['goal_type']}, Target Weight: {state['target_weight']}kg in {state['target_days']} days
    - Nutrition Plan: {state['nutrition_plan']}
    - Additional Notes: {state['user_notes']}
    
    Provide:
    1. Weekly workout schedule
    2. Exercise types and intensity
    3. Progressive overload plan
    4. Recovery recommendations
    5. Alternative exercises for equipment limitations
    """
    
    # Here you would call your LLM
    workout_plan = {
        "weekly_schedule": {
            "monday": "Upper body strength training",
            "tuesday": "Cardio - 30min moderate intensity",
            "wednesday": "Lower body strength training",
            "thursday": "Active recovery - yoga/stretching",
            "friday": "Full body circuit training",
            "saturday": "Cardio - 45min low intensity",
            "sunday": "Rest day"
        },
        "progression": "Weekly progression guidelines...",
        "recovery": "Recovery and rest recommendations..."
    }
    
    state["workout_plan"] = workout_plan
    state["current_step"] = "complete"
    return state

def handle_chat_query(state: FitnessAppState) -> FitnessAppState:
    """Node to handle general user queries in chat"""
    print(f"💬 Processing your question: {state['chat_query']}")
    
    chat_prompt = f"""
    User context:
    - Profile: Age {state['age']}, {state['gender']}, {state['height']}cm, {state['weight']}kg
    - Activity Level: {state['activity_level']}
    - Goals: {state['goal_type']}, target {state['target_weight']}kg in {state['target_days']} days
    - User Notes: {state['user_notes']}
    - Current Nutrition Plan: {state['nutrition_plan']}
    - Current Workout Plan: {state['workout_plan']}
    
    User Question: {state['chat_query']}
    
    Provide a helpful, personalized response considering their profile and plans.
    """
    
    # Here you would call your LLM
    response = f"Based on your profile and current plans, here's my response to your question about: {state['chat_query']}"
    
    # Add to chat history
    if not state["chat_messages"]:
        state["chat_messages"] = []
    
    state["chat_messages"].append({
        "user": state["chat_query"],
        "assistant": response
    })
    
    state["chat_response"] = response
    state["chat_query"] = None
    return state

# Helper functions
def calculate_bmr(height: float, weight: float, age: int, gender: str) -> float:
    """Calculate Basal Metabolic Rate using Mifflin-St Jeor equation"""
    if gender.lower() == "male":
        return 10 * weight + 6.25 * height - 5 * age + 5
    else:
        return 10 * weight + 6.25 * height - 5 * age - 161

def calculate_daily_calories(bmr: float, activity_level: str) -> float:
    """Calculate daily calorie needs based on activity level"""
    multipliers = {
        "sedentary": 1.2,
        "lightly_active": 1.375,
        "moderately_active": 1.55,
        "very_active": 1.725,
        "extremely_active": 1.9
    }
    return bmr * multipliers.get(activity_level.lower(), 1.2)

def adjust_calories_for_goal(daily_calories: float, goal_type: str, target_weight: float, current_weight: float, target_days: int) -> float:
    """Adjust calories based on user goals"""
    if goal_type == "weight_loss":
        # Safe deficit: 0.5-1 kg per week = 500-1000 cal deficit per day
        weekly_loss = (current_weight - target_weight) / (target_days / 7)
        deficit = min(weekly_loss * 1000, 1000)  # Max 1000 cal deficit
        return daily_calories - deficit
    elif goal_type == "muscle_gain":
        return daily_calories + 300  # Moderate surplus
    else:
        return daily_calories  # Maintenance

# Conditional Edge Functions
def should_edit_nutrition(state: FitnessAppState) -> str:
    """Determine if user wants to edit nutrition plan"""
    if state.get("edit_request"):
        return "edit_nutrition"
    elif state.get("nutrition_plan_approved"):
        return "generate_workout"
    else:
        return "nutrition_review"

def has_user_input_pending(state: FitnessAppState) -> str:
    """Check if waiting for user input"""
    if state.get("user_input_pending"):
        return "wait_for_input"
    else:
        return "continue"

def is_chat_query(state: FitnessAppState) -> str:
    """Check if this is a chat query"""
    if state.get("chat_query"):
        return "handle_chat"
    else:
        return "continue_workflow"

# Build the LangGraph
def create_fitness_app_graph():
    """Create and configure the LangGraph workflow"""
    
    # Initialize the graph
    workflow = StateGraph(FitnessAppState)
    
    # Add nodes
    workflow.add_node("collect_personal_info", collect_personal_info)
    workflow.add_node("collect_goals", collect_goals)
    workflow.add_node("collect_notes", collect_notes)
    workflow.add_node("generate_nutrition_plan", generate_nutrition_plan)
    workflow.add_node("handle_nutrition_edit", handle_nutrition_edit)
    workflow.add_node("generate_workout_plan", generate_workout_plan)
    workflow.add_node("handle_chat_query", handle_chat_query)
    
    # Set entry point
    workflow.set_entry_point("collect_personal_info")
    
    # Add edges
    workflow.add_edge("collect_personal_info", "collect_goals")
    workflow.add_edge("collect_goals", "collect_notes")
    workflow.add_edge("collect_notes", "generate_nutrition_plan")
    
    # Add conditional edges
    workflow.add_conditional_edges(
        "generate_nutrition_plan",
        should_edit_nutrition,
        {
            "edit_nutrition": "handle_nutrition_edit",
            "generate_workout": "generate_workout_plan",
            "nutrition_review": "generate_nutrition_plan"
        }
    )
    
    workflow.add_conditional_edges(
        "handle_nutrition_edit",
        should_edit_nutrition,
        {
            "generate_workout": "generate_workout_plan",
            "nutrition_review": "generate_nutrition_plan"
        }
    )
    
    # Chat functionality can be accessed from any completed state
    workflow.add_conditional_edges(
        "generate_workout_plan",
        is_chat_query,
        {
            "handle_chat": "handle_chat_query",
            "continue_workflow": END
        }
    )
    
    # Chat can loop back to itself or end
    workflow.add_conditional_edges(
        "handle_chat_query",
        is_chat_query,
        {
            "handle_chat": "handle_chat_query",
            "continue_workflow": END
        }
    )
    
    # Add memory for conversation persistence
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    
    return app

# Example usage and testing
def main():
    """Main function to demonstrate the workflow"""
    
    # Create the application
    app = create_fitness_app_graph()
    
    # Initialize state
    initial_state = FitnessAppState(
        height=None,
        weight=None,
        age=None,
        gender=None,
        activity_level=None,
        goal_type=None,
        target_weight=None,
        target_days=None,
        user_notes=None,
        nutrition_plan=None,
        nutrition_plan_approved=None,
        workout_plan=None,
        current_step="start",
        user_input_pending=False,
        edit_request=None,
        chat_messages=[],
        chat_query=None,
        chat_response=None,
        error_message=None
    )
    
    # Configuration for conversation persistence
    config = RunnableConfig(configurable={"thread_id": "user_123"})
    
    # Example: Start the workflow
    print("🚀 Starting Fitness AI Application")
    print("=" * 50)
    
    # Step 1: Collect personal info
    result = app.invoke(initial_state, config)
    print(f"Current step: {result['current_step']}")
    
    # Example of updating state with user input
    updated_state = result.copy()
    updated_state.update({
        "height": 175.0,
        "weight": 80.0,
        "age": 28,
        "gender": "male",
        "activity_level": "moderately_active",
        "user_input_pending": False
    })
    
    # Continue workflow...
    print("\n" + "=" * 50)
    print("Workflow created successfully!")
    print("Key features:")
    print("✅ Sequential data collection")
    print("✅ AI-powered plan generation")
    print("✅ Interactive editing capability")
    print("✅ Contextual chat support")
    print("✅ State persistence")
    print("✅ Error handling")

if __name__ == "__main__":
    main()